# Utility Notebook

This notebook serves as a platform to generate the data from the experimental image sequence. It is organized in three distinct parts. We first manipulate images on the notebook to determine the optimal image processing parameters. We then segment the images and track the particles. Finally, we post-process the data and save it.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import file management tools. 
import os 
import shutil
import pandas

# Image processing tools
import skimage
import skimage.io
import skimage.filters
import skimage.morphology
import skimage.segmentation
import skimage.measure
import skimage.feature
import scipy.ndimage
import skimage.data
from skimage import img_as_float
import scipy.misc
from PIL import Image

import sys

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

We then import the specific functions.

In [ ]:
# Segmentation and feature extractionm
import grayscale_analysis
import grayscale_segmentation

# Track creation
import PreferenceCreation
import GaleShapley
import Linking

# Post-processing
import LinkLooseEnds
import MassMerge

# Feature extraction
import DisplacementProperties

## 1) Parameter estimation

The first step consists in importing the image stack.

In [ ]:
file_dir = 'FILE FOLDER'
data_dir = 'EXPERIMENT NUMBER'
seg_dir = 'LOCATION OF THE SEGMENTED IMAGES'
save_dir = 'LOCATION OF THE EXTRACTED DATA'

In [ ]:
rep_raw = file_dir + data_dir
rep_data = save_dir + seg_dir + data_dir + '/image folder'
rep_prop = save_dir + seg_dir + data_dir + '/property folder'
rep_link = save_dir + seg_dir + data_dir + '/link folder'
rep_pref = save_dir + seg_dir + data_dir + '/preferences folder'
rep_link_prop = save_dir + seg_dir + data_dir

The values below determine the properties of the segmentation algorithm. These values have been optimized through trial and error on cell slide images. They are to be adapted locally for each image.

The orientation type refers to the procedure chosen. 

 - 'naive': we measure the cell orientation by fitting an ellipse with the cell as it has been segmented.
 - 'luminosity': we measure the cell orientation by fitting an ellipse with the 20% most luminous pixels.
 
The 'invert' variable refers to the relative color of the cells to the background. If 'True', we invert the pixel values in order to have cells with lower luminosty values than the background.

In [ ]:
cells = 'A NUMBER BETWEEN 0 AND 1'
background = 'A NUMBER BETWEEN 0 AND 1'

min_size = 'MINIMUM CELL SIZE'
min_dist = 'MINIMUM DISTANCE BETWEEN TWO CELLS'

orientation = 'naive'/'luminosty'
invert = True/False

In [ ]:
n = 'IMAGE NUMBER'
threshold = 'LUMINOSITY THRESHOLD'
sigma = 'AN INT'

os.chdir(rep_raw)
file_list  = sorted(os.listdir(rep_raw))
file_list.sort(key=len, reverse=False)

im = grayscale_segmentation._contrast_enhance(file_list[n], rep_raw, threshold, sigma, invert)
fig = plt.figure(figsize=(5,5))
plt.imshow(im, cmap = plt.cm.gray)

Once the choice of the parameters is settled we can move on to the study of the images.

In [ ]:
seg = grayscale_segmentation._image_segment_none(im, cells, background, min_size, min_dist)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(im, cmap = plt.cm.gray)
plt.imshow(seg, cmap = plt.cm.spectral, alpha = 0.3)
plt.colorbar()

When the parameters have been chosen, we launch the segmentation algorithm. This program will create a folder where all the segmented images will be kept for future use purposes in a '.npy' array.

In [ ]:
grayscale_segmentation._get_stack_grayscale(rep_raw, threshold, sigma, cells, 
                         background, min_size, min_dist, 'naive', rep_data, invert)

## 2) Image analysis: property extraction and linking

Now that we have generated and saved the segmented images we can move on to analysing them. This step should be the most time-consuming one. If the option "orientation" is set to False, the average time per frame is around 1'30". This is linear in the number of particles detected per frame.

In [ ]:
results = grayscale_analysis._get_properties_stack(rep_data, rep_prop, rep_raw, orientation)

Thanks to the function above we have created data frames which contain all the measured features from the different cells. Once these are extracted, we have to link cells between each other. This is first achieved by creating the preferred potential links for each cell, then, with help of the modified Gale-Shapley algorithm, we link each cell to its counterpart in the next frame.

In [ ]:
r = 'DISTANCE IN PIXELS'
cost = 'MAX VALUE OF THE COST FUNCTION'
cost_type = 'distance'

In [ ]:
PreferenceCreation._get_preferences_stack(r, min_size, cost, cost_type, rep_prop, rep_pref)

In [ ]:
sys.setrecursionlimit(5000)

GaleShapley._get_linking(rep_pref, rep_link)

We generate trajectories by linking the measured properties for each cell to each other. We generate a new data frame that is saved.

In [ ]:
df = Linking._get_prop_frame(rep_prop, rep_link,rep_link_prop)

In [ ]:
current_dir = os.getcwd()
os.chdir(rep_link_prop)
df = pandas.read_csv('PropertyFrame')
os.chdir(current_dir)

## 3) Post-processing

We then correct for the major causes of mis-linking: one cell segmented as two and the non-segmentation of a cell over several frames. The parameters have to be adapted for each experiment.

In [ ]:
dist = 'MAX LINKING DISTANCE'
pc = 'MASS CHANGE PERCENTAGE'
# defines the level of precision that the sum of the two particles has to have
# with regard to the individual cell masses.

ef, df = MassMerge._mass_merger(df, dist, pc)

In [ ]:
memory = 'MEMORY'
dist = 'MAX LINKING DISTANCE'

ef, df = LinkLooseEnds._link_ends(df, memory, dist)

In [ ]:
os.chdir(rep_link_prop)
df.to_csv('PropertyFrameOrdered', index = False)

## 4) Final feature extraction

We extract the information related to cell movement.

In [ ]:
ResultFrame = DisplacementProperties._get_dynamic_prop(dfp)

In [ ]:
current_dir = os.getcwd()
os.chdir(rep_link_prop)
ResultFrame.to_csv('PropertyFrameOrdered', index = False)
os.chdir(current_dir)